In [2]:
!pip install nltk -U

     |████████████████████████████████| 1.5MB 28.6MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [3]:
from nltk.translate.bleu_score import *
from nltk.translate.meteor_score import *
import pandas as pd
import ast

In [4]:
import os
#os.system("git clone https://github.com/soerenetler/squad-question-generation.git")
#os.system("git clone https://github.com/soerenetler/spacy_target_answer.git")
#os.system("git clone https://github.com/soerenetler/crf_target_answer.git")

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
path_to_folder = "/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/squad/question_answer/"
df = pd.read_csv(path_to_folder+"test.csv")
df["answer_sentence_token"] = [ast.literal_eval(t) for t in df["answer_sentence_token"]]
df["question_token"] = [ast.literal_eval(t) for t in df["question_token"]]

In [6]:
import unicodedata
def unicode_to_ascii(s):
    s = s.lower().strip()
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')
    
df["question_token"] = [[unicode_to_ascii(t)for t in question] for question in df["question_token"]]
df["answer_sentence_token"] = [[unicode_to_ascii(t)for t in question] for question in df["answer_sentence_token"]]

In [7]:
df.head()

,Unnamed: 0,text_title,question_text,question_token,question_pos,question_tag,question_parse_tree,answer_sentence,answer_sentence_token,answer_paragraph,masked_answer_paragraph,masked_answer_sentence,answer,unanswerable
0,0,Normans,In what country is Normandy located?,"[in, what, country, is, normandy, located, ?]","['ADP', 'DET', 'NOUN', 'AUX', 'PROPN', 'VERB',...","['IN', 'WDT', 'NN', 'VBZ', 'NNP', 'VBN', '.']",NaN,The Normans (Norman: Nourmands; French: Norman...,"[the, normans, (, norman, :, nourmands, ;, fre...",The Normans (Norman: Nourmands; French: Norman...,"['The', 'Normans', '(', 'Norman', ':', 'Nourma...","['the', 'normans', '(', 'norman', ':', 'nourma...",France,False
1,1,Normans,In what country is Normandy located?,"[in, what, country, is, normandy, located, ?]","['ADP', 'DET', 'NOUN', 'AUX', 'PROPN', 'VERB',...","['IN', 'WDT', 'NN', 'VBZ', 'NNP', 'VBN', '.']",NaN,The Normans (Norman: Nourmands; French: Norman...,"[the, normans, (, norman, :, nourmands, ;, fre...",The Normans (Norman: Nourmands; French: Norman...,"['The', 'Normans', '(', 'Norman', ':', 'Nourma...","['the', 'normans', '(', 'norman', ':', 'nourma...",France,False
2,2,Normans,In what country is Normandy located?,"[in, what, country, is, normandy, located, ?]","['ADP', 'DET', 'NOUN', 'AUX', 'PROPN', 'VERB',...","['IN', 'WDT', 'NN', 'VBZ', 'NNP', 'VBN', '.']",NaN,The Normans (Norman: Nourmands; French: Norman...,"[the, normans, (, norman, :, nourmands, ;, fre...",The Normans (Norman: Nourmands; French: Norman...,"['The', 'Normans', '(', 'Norman', ':', 'Nourma...","['the', 'normans', '(', 'norman', ':', 'nourma...",France,False
3,3,Normans,In what country is Normandy located?,"[in, what, country, is, normandy, located, ?]","['ADP', 'DET', 'NOUN', 'AUX', 'PROPN', 'VERB',...","['IN', 'WDT', 'NN', 'VBZ', 'NNP', 'VBN', '.']",NaN,The Normans (Norman: Nourmands; French: Norman...,"[the, normans, (, norman, :, nourmands, ;, fre...",The Normans (Norman: Nourmands; French: Norman...,"['The', 'Normans', '(', 'Norman', ':', 'Nourma...","['the', 'normans', '(', 'norman', ':', 'nourma...",France,False
4,4,Normans,When were the Normans in Normandy?,"[when, were, the, normans, in, normandy, ?]","['ADV', 'AUX', 'DET', 'PROPN', 'ADP', 'PROPN',...","['WRB', 'VBD', 'DT', 'NNPS', 'IN', 'NNP', '.']",NaN,The Normans (Norman: Nourmands; French: Norman...,"[the, normans, (, norman, :, nourmands, ;, fre...",The Normans (Norman: Nourmands; French: Norman...,"['The', 'Normans', '(', 'Norman', ':', 'Nourma...","['the', 'normans', '(', 'norman', ':', 'nourma...",10th and 11th centuries,False


In [49]:
sentence_bleu(["what is the original meaning of the word norman ?".split(" ")], "what is the name of the anglo word ?".split(" "), weights=(1,0,0,0))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning:


The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()



0.6959861353000654

## Compare Question to target sentence as reference 

In [8]:
df["sentence_question_bleu"] =[sentence_bleu([question], sentence) for sentence, question in zip(df["question_token"], df["answer_sentence_token"])]
print("BLEU: {}".format(corpus_bleu([[question] for question in df["question_token"]], df["answer_sentence_token"], weights=(.25,.25,.25,.25))*100))
print("BLEU1: {}".format(corpus_bleu([[question] for question in df["question_token"]], df["answer_sentence_token"], weights=(1,0,0,0))*100))
print("BLEU2: {}".format(corpus_bleu([[question] for question in df["question_token"]], df["answer_sentence_token"], weights=(0,1,0,0))*100))
print("BLEU3: {}".format(corpus_bleu([[question] for question in df["question_token"]], df["answer_sentence_token"], weights=(0,0,1,0))*100))
print("BLEU4: {}".format(corpus_bleu([[question] for question in df["question_token"]], df["answer_sentence_token"], weights=(0,0,0,1))*100))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

BLEU: 5.926391434549317
BLEU1: 16.948862714606626
BLEU2: 7.546587218865038
BLEU3: 4.089705663004026
BLEU4: 2.3581863511081025


In [9]:
import plotly.express as px
px.histogram(df, x="sentence_question_bleu")

# Compare to predicted Question

In [10]:
pred_file = "/content/gdrive/MyDrive/mt-qg-data/00_models/qg_attention/squad/dataset_squad-target_length_20-input_length_60-vocab_input_45000-max_vocab_targ_28000-epochs_15-units_512-batch_64-layer_1-dropout_0.3-pretrained_True-bidirectional_True-answer_units_256-/test.txt"

In [11]:
pred_questions= []
with open(pred_file, "r") as f:
  for line in f:
    token = line.strip().split(" ")
    try:
      end_index = token.index("<end>")
    except ValueError:
      end_index = len(token)
    pred_questions.append(token[:end_index])

In [45]:
print(pred_questions[30])
print(df["question_token"][30])

['what', 'is', 'the', 'name', 'of', 'the', 'word', 'word', 'name', 'for', 'the', 'anglo', 'word', '?']
['what', 'is', 'the', 'original', 'meaning', 'of', 'the', 'word', 'norman', '?']


In [13]:
df["sentence_question_bleu"] =[sentence_bleu([question], sentence) for sentence, question in zip(df["question_token"], pred_questions)]
print("BLEU: {}".format(corpus_bleu([[question] for question in df["question_token"]], pred_questions, weights=(.25,.25,.25,.25))*100))
print("BLEU1: {}".format(corpus_bleu([[question] for question in df["question_token"]], pred_questions, weights=(1,0,0,0))*100))
print("BLEU2: {}".format(corpus_bleu([[question] for question in df["question_token"]], pred_questions, weights=(0,1,0,0))*100))
print("BLEU3: {}".format(corpus_bleu([[question] for question in df["question_token"]], pred_questions, weights=(0,0,1,0))*100))
print("BLEU4: {}".format(corpus_bleu([[question] for question in df["question_token"]], pred_questions, weights=(0,0,0,1))*100))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning:


The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning:


The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning:


The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()



BLEU: 4.431624352136189
BLEU1: 30.782574353193848
BLEU2: 6.408831033835552
BLEU3: 2.3452537720326205
BLEU4: 0.8336392586726092


# BLEU divided by answer length

In [32]:
df["pred_question"] = pred_questions
df["ans_length"] = [len(a.split(" ")) for a in df["answer"]]

all_bleu_dict = {"BLEU":[],
                 "BLEU1": [],
                 "BLEU2": [],
                 "BLEU3": [],
                 "BLEU4": []}

weight_dict = {"BLEU":(.25,.25,.25,.25),
                "BLEU1": (1,0,0,0),
                "BLEU2": (0,1,0,0),
                "BLEU3": (0,0,1,0),
                "BLEU4": (0,0,0,1)}
names = ["BLEU", "BLEU1", "BLEU2", "BLEU3", "BLEU4"]

ans_lengths = [1,2,3,4,5,6,7,8]

for name in names:
  for ans_length in ans_lengths:
    tmp_df = df[df["ans_length"]==ans_length]
    all_bleu_dict[name].append(corpus_bleu([[question] for question in tmp_df["question_token"]], tmp_df["pred_question"], weights=weight_dict[name])*100)

In [42]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=6,subplot_titles=names)
fig.update_yaxes(title_text="Score", row=1, col=1)
for i, item in enumerate(all_bleu_dict.items(),1):
  key, value=item
  fig.add_trace(
      go.Scatter(x=list(range(1,9)), y=value),
      row=1, col=i
  )
  
  fig.update_xaxes(title_text="Answer Length", row=1, col=i)

fig.update_layout(height=600, width=1600, title_text="BLEU Score by Answer Length")
fig.show()

# First Token Confusion matrix

In [50]:
from sklearn.metrics import confusion_matrix
from collections import Counter
import plotly.graph_objects as go

In [51]:
def get_interrogative_phrase(question_token):
  return question_token[0]

def plot_confusion_matrix(y_true, y_pred):
    most_common = Counter(y_true).most_common(100)
    most_common_labels = [e[0] for e in most_common]
    my_confusion_matrix = confusion_matrix(y_true, y_pred, labels=most_common_labels, normalize="true")

    layout = {
        "title": "Confusion Matrix", 
        "xaxis": {"title": "Predicted value"}, 
        "yaxis": {"title": "Real value"}
    }

    fig = go.Figure(data=go.Heatmap(z=my_confusion_matrix[:,:],
                                    x=most_common_labels[:5],
                                    y=most_common_labels[:5],
                                    hoverongaps=False),
                    layout=layout)
    fig.show()

pred_wh = [get_interrogative_phrase(question) for question in pred_questions]
true_wh = [get_interrogative_phrase(question) for question in df["question_token"]]

In [52]:
plot_confusion_matrix(true_wh, pred_wh)

In [51]:
from sklearn.metrics import classification_report
most_common = Counter(true_wh).most_common(100)
most_common_labels = [e[0] for e in most_common]
classification_report(true_wh, pred_wh, labels=most_common_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



'                precision    recall  f1-score   support\n\n          what       0.51      0.76      0.61      9297\n           how       0.36      0.36      0.36      1922\n           who       0.27      0.40      0.32      1572\n          when       0.30      0.32      0.31      1333\n            in       0.21      0.05      0.09       828\n         where       0.22      0.10      0.14       763\n         which       0.05      0.01      0.01       729\n           the       0.00      0.00      0.00       520\n           why       0.00      0.00      0.00       358\n            by       0.00      0.00      0.00       112\n         along       0.00      0.00      0.00       101\n            if       0.00      0.00      0.00        95\n            to       0.00      0.00      0.00        74\n     according       0.43      0.04      0.08        69\n         other       0.00      0.00      0.00        69\n            at       0.00      0.00      0.00        69\n       besides       0.00   